In [1]:
from signals_in_the_noise.preprocessing.data_gse161529 import GSE161529

gse = GSE161529()
# adata = next(iter(gse.objects.values()))
# adata

2025-07-04 21:33:59,938 [INFO] signals_in_the_noise.utilities.tenx_genomics: Reading C:\Users\silly\GitHub\signals-in-the-noise\data\GSE161529_adata_cache\GSM4909253_N-PM0092-Total.h5ad as AnnData object.
2025-07-04 21:34:00,019 [INFO] signals_in_the_noise.utilities.tenx_genomics: Reading C:\Users\silly\GitHub\signals-in-the-noise\data\GSE161529_adata_cache\GSM4909254_N-PM0019-Total.h5ad as AnnData object.
2025-07-04 21:34:00,099 [INFO] signals_in_the_noise.utilities.tenx_genomics: Reading C:\Users\silly\GitHub\signals-in-the-noise\data\GSE161529_adata_cache\GSM4909255_N-N280-Epi.h5ad as AnnData object.
2025-07-04 21:34:00,135 [INFO] signals_in_the_noise.utilities.tenx_genomics: Reading C:\Users\silly\GitHub\signals-in-the-noise\data\GSE161529_adata_cache\GSM4909256_N-PM0095-Epi.h5ad as AnnData object.
2025-07-04 21:34:00,219 [INFO] signals_in_the_noise.utilities.tenx_genomics: Reading C:\Users\silly\GitHub\signals-in-the-noise\data\GSE161529_adata_cache\GSM4909257_N-PM0095-Total.h5ad 

In [ ]:
adata = list(gse.objects.items())[66][1]

In [ ]:
import scanpy as sc
# objs = list(gse.objects.values())
# adata = objs[2]
adata.var['mt'] = adata.var_names.str.upper().str.startswith('MT-')
sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], inplace=True)


In [ ]:
num_genes_lower_bound = adata.uns['qc_genes_lower']
num_genes_upper_bound = adata.uns['qc_genes_upper']
mito_upper_bound = adata.uns['qc_mito_upper']
total_count_upper_bound = adata.uns['qc_total_upper']

adata.obs['is_low_num_genes'] = (adata.obs['n_genes_by_counts'] <= num_genes_lower_bound).astype(int)
adata.obs['is_high_num_genes'] = (adata.obs['n_genes_by_counts'] >= num_genes_upper_bound).astype(int)
adata.obs['is_high_mito'] = (adata.obs['pct_counts_mt']/100 >= mito_upper_bound).astype(int)
adata.obs['is_high_total_count'] = (adata.obs['total_counts'] >= total_count_upper_bound).astype(int)

adata.obs['is_noise'] = (
    adata.obs['is_low_num_genes'] |
    adata.obs['is_high_num_genes'] |
    adata.obs['is_high_mito'] |
    adata.obs['is_high_total_count']
).astype(int)

print(bool(adata[adata.obs['is_noise'] == 0, :].shape[0] == adata.uns['num_cells_after']), adata[adata.obs['is_noise'] == 0, :].shape[0], int(adata.uns['num_cells_after']))
print(adata[adata.obs['is_low_num_genes']==1, :].shape[0], adata[adata.obs['is_high_num_genes']==1, :].shape[0], adata[adata.obs['is_high_mito']==1, :].shape[0], adata[adata.obs['is_high_total_count']==1, :].shape[0], 4966-4443)

In [ ]:
# num_genes_lower_bound, num_genes_upper_bound, mito_upper_bound, total_count_upper_bound
# adata.obs[['is_noise', 'is_low_num_genes', 'is_high_num_genes', 'is_high_mito', 'is_high_total_count']]
adata.uns